In [ ]:
import numpy as np

The coordinates for the para, meta, and ortho intermediates follow

In [ ]:
# coordinates for para intermediate
para_coords = [
 "C                 -0.80658313    1.22973465    0.03041801",
 "C                  0.56153576    1.23725234    0.01622618",
 "C                  1.22915389    0.01001055    0.01220575",
 "H                 -1.36676923    2.15803094    0.04420367",
 "H                  1.14116413    2.14927050    0.01037697",
 "N                  2.71357475    0.03144573   -0.00289824",
 "O                  3.28013247   -1.09741954   -0.00254733",
 "O                  3.24714953    1.17621948   -0.01252002",
 "C                 -0.77042978   -1.26805414    0.04039660",
 "H                 -1.30353926   -2.21202933    0.06122375",
 "C                  0.59726287   -1.23605918    0.02634378",
 "H                  1.20308359   -2.13089607    0.02793117",
 "C                 -1.56287141   -0.03049318    0.01040538",
 "H                 -2.41148563   -0.03994459    0.70143946",
 "Br                -2.40993182   -0.04931830   -1.82359612"]

# coordinates for meta intermediate
meta_coords = [
 "C                  0.02949981    1.33972592    0.06817723",
 "C                  1.43483278    1.28667967    0.00635313",
 "C                  2.11179024    0.05106117   -0.00544138",
 "C                  1.44506636   -1.13720058    0.03116583",
 "C                 -0.68793171    0.16822220    0.10995314",
 "H                 -0.47126997    2.29839666    0.07811355",
 "H                  2.02732783    2.19651728   -0.03220624",
 "H                  1.98966526   -2.07643217    0.02318494",
 "H                 -1.77163480    0.18040547    0.15819632",
 "N                  3.58635895    0.05097292   -0.06745286",
 "O                  4.14711759   -1.05966097   -0.08807849",
 "O                  4.14497859    1.16390951   -0.09010823",
 "C                 -0.02361177   -1.14582791    0.08353483",
 "H                 -0.43674996   -1.87247364    0.78889576",
 "Br                -0.53591638   -1.86972195   -1.74078671"
]

# coordinates for ortho intermediate
ortho_coords = [
 "C                  0.51932475    1.23303451   -0.03194925",
 "C                  1.94454413    1.26916358   -0.03672882",
 "C                  2.62037793    0.09283428   -0.02499003",
 "C                 -0.19603352    0.03013062    0.00102732",
 "H                 -0.02069420    2.17423764   -0.04336646",
 "H                  2.48281698    2.20891057   -0.03611879",
 "H                 -1.27770137    0.03990295    0.01166953",
 "N                  4.09213475    0.09594076    0.03662979",
 "O                  4.63930696   -1.02169275    0.14459220",
 "O                  4.66489883    1.19839699   -0.02327545",
 "C                  0.49428518   -1.16712649    0.02099746",
 "H                 -0.03251071   -2.11492669    0.05447935",
 "C                  1.96291176   -1.21653219   -0.02111314",
 "H                  2.44359113   -1.96306433    0.61513886",
 "Br                 2.17304025   -1.94912156   -1.90618750"
]


## Parsing Molecular Coordinates

We need to convert the coordinate strings into a structured format that's easy to work with. Our goal is to organize coordinates by atom type (C, N, O, H, Br) so we can:
- Calculate the center of mass (using all atoms)
- Define molecular orientation vectors (using specific atoms like the nitrogen and carbons)
- Apply transformations during the simulation

### Input Format
Each coordinate string has the format:
```
"ATOM_LABEL    x_coord    y_coord    z_coord"
```
For example: `"C  -0.80658313  1.22973465  0.03041801"`

### Output Format
The parser creates a dictionary where:
- **Keys**: Atom types (e.g., 'C', 'N', 'O', 'H', 'Br')
- **Values**: NumPy arrays of shape `(n_atoms, 3)` containing coordinates

This allows us to easily access all carbons with `parsed_dict['C']` or the nitrogen with `parsed_dict['N']`.

### Parse Coordinate Function

The `parse_coordinates()` function converts coordinate strings into organized NumPy arrays:

**Steps:**
1. **Split each line** into atom label and coordinate values
2. **Group coordinates by atom type**
3. **Convert to NumPy arrays** for efficient numerical operations

**Why organize by atom type?**
- We need all carbons to define the benzene ring plane
- We need the nitrogen to define the C-N bond direction
- We can easily calculate mass-weighted center of mass by atom type
- Makes it simple to apply transformations to the entire molecule

### Parsing All Three Isomers

We parse coordinates for all three nitrobenzene isomers:
- **meta**: NO₂ group at meta position (1,3-substitution)
- **ortho**: NO₂ group at ortho position (1,2-substitution)  
- **para**: NO₂ group at para position (1,4-substitution)

The output shows the coordinates organized by atom type. For example, `para_parsed['C']` contains a 6×3 array with the (x,y,z) coordinates of all 6 carbon atoms in the benzene ring.

In [ ]:
def parse_coordinates(coord_list):
    parsed_dict = {}
    for line in coord_list:
        parts = line.split()
        atom_label = parts[0]
        # Coordinates start from the second element
        coords = np.array([float(parts[1]), float(parts[2]), float(parts[3])])

        if atom_label not in parsed_dict:
            parsed_dict[atom_label] = []
        parsed_dict[atom_label].append(coords)

    # Convert lists of arrays to single numpy arrays for each atom type
    for atom, coords_list in parsed_dict.items():
        parsed_dict[atom] = np.array(coords_list)
    return parsed_dict

# Parse each of the coordinate lists
meta_parsed = parse_coordinates(meta_coords)
ortho_parsed = parse_coordinates(ortho_coords)
para_parsed = parse_coordinates(para_coords)

print("Parsed meta_coords:")
for atom, coords in meta_parsed.items():
    print(f"  {atom}:\n{coords}")

print("\nParsed ortho_coords:")
for atom, coords in ortho_parsed.items():
    print(f"  {atom}:\n{coords}")

print("\nParsed para_coords:")
for atom, coords in para_parsed.items():
    print(f"  {atom}:\n{coords}")

Parsed meta_coords:
  C:
[[ 0.02949981  1.33972592  0.06817723]
 [ 1.43483278  1.28667967  0.00635313]
 [ 2.11179024  0.05106117 -0.00544138]
 [ 1.44506636 -1.13720058  0.03116583]
 [-0.68793171  0.1682222   0.10995314]
 [-0.02361177 -1.14582791  0.08353483]]
  H:
[[-0.47126997  2.29839666  0.07811355]
 [ 2.02732783  2.19651728 -0.03220624]
 [ 1.98966526 -2.07643217  0.02318494]
 [-1.7716348   0.18040547  0.15819632]
 [-0.43674996 -1.87247364  0.78889576]]
  N:
[[ 3.58635895  0.05097292 -0.06745286]]
  O:
[[ 4.14711759 -1.05966097 -0.08807849]
 [ 4.14497859  1.16390951 -0.09010823]]
  Br:
[[-0.53591638 -1.86972195 -1.74078671]]

Parsed ortho_coords:
  C:
[[ 5.19324750e-01  1.23303451e+00 -3.19492500e-02]
 [ 1.94454413e+00  1.26916358e+00 -3.67288200e-02]
 [ 2.62037793e+00  9.28342800e-02 -2.49900300e-02]
 [-1.96033520e-01  3.01306200e-02  1.02732000e-03]
 [ 4.94285180e-01 -1.16712649e+00  2.09974600e-02]
 [ 1.96291176e+00 -1.21653219e+00 -2.11131400e-02]]
  H:
[[-0.0206942   2.17423764

## Finding the Carbon Bonded to Nitrogen

In nitrobenzene, the nitrogen atom is directly bonded to one of the six carbons in the benzene ring. To define the **xi_hat** vector (C-N bond direction), we need to identify which carbon this is.

**Strategy:** The bonded carbon will be the one **closest** to the nitrogen atom. We can find it by:
1. Computing the distance from N to each carbon atom
2. Finding the index of the minimum distance

This approach works regardless of which isomer (meta, ortho, para) we're analyzing.

# Key ideas
1. Define a vector $\hat{x}_i$ which is a unit vector oriented along the N atom.  In essence, we take the vector that defines the position of the N atom and normalize it to define $\hat{x}_i$
2. Define a vector $\hat{z}_i$ which is a unit vector oriented out of plane of the benzene ring.  This can be defined from the cross product of any two vectors defining carbon atoms on the benzene ring.

In [ ]:
def find_N_bonded_carbon_index(parsed_coords):
    """
    Find the index of the carbon atom bonded to nitrogen.

    The bonded carbon is identified as the carbon closest to the nitrogen atom.

    Parameters:
    -----------
    parsed_coords : dict
        Dictionary with atom types as keys and coordinate arrays as values
        Must contain 'N' and 'C' keys

    Returns:
    --------
    int : Index of the carbon atom bonded to nitrogen (0-indexed)
    """
    # Get nitrogen coordinates (should be only one N atom)
    N_coord = parsed_coords['N'][0] if parsed_coords['N'].ndim > 1 else parsed_coords['N']

    # Get all carbon coordinates
    C_coords = parsed_coords['C']

    # Calculate distances from N to each carbon
    distances = np.linalg.norm(C_coords - N_coord, axis=1)

    # Find index of minimum distance
    bonded_carbon_index = np.argmin(distances)

    return bonded_carbon_index

# Find the bonded carbon for each isomer
meta_C_index = find_N_bonded_carbon_index(meta_parsed)
ortho_C_index = find_N_bonded_carbon_index(ortho_parsed)
para_C_index = find_N_bonded_carbon_index(para_parsed)

print("Carbon bonded to nitrogen:")
print(f"  meta isomer:  C index {meta_C_index}")
print(f"  ortho isomer: C index {ortho_C_index}")
print(f"  para isomer:  C index {para_C_index}")

# Verify by showing the C-N distances
print("\nVerification - distances from N to each carbon:")
for name, parsed in [('meta', meta_parsed), ('ortho', ortho_parsed), ('para', para_parsed)]:
    N_coord = parsed['N'][0]
    C_coords = parsed['C']
    distances = np.linalg.norm(C_coords - N_coord, axis=1)
    print(f"\n{name} isomer:")
    for i, dist in enumerate(distances):
        marker = " <-- BONDED" if i == find_N_bonded_carbon_index(parsed) else ""
        print(f"  C{i}: {dist:.4f} Å{marker}")

Carbon bonded to nitrogen:
  meta isomer:  C index 2
  ortho isomer: C index 2
  para isomer:  C index 2

Verification - distances from N to each carbon:

meta isomer:
  C0: 3.7856 Å
  C1: 2.4822 Å
  C2: 1.4759 Å <-- BONDED
  C3: 2.4508 Å
  C4: 4.2796 Å
  C5: 3.8062 Å

ortho isomer:
  C0: 3.7500 Å
  C1: 2.4483 Å
  C2: 1.4730 Å <-- BONDED
  C3: 4.2888 Å
  C4: 3.8131 Å
  C5: 2.5019 Å

para isomer:
  C0: 3.7187 Å
  C1: 2.4669 Å
  C2: 1.4847 Å <-- BONDED
  C3: 3.7187 Å
  C4: 2.4670 Å
  C5: 4.2769 Å


## Computing Molecular Orientation Vectors

To track how the nitrobenzene molecule rotates in the electric field, we need to define a molecular reference frame. We'll use two key vectors:

1. **x_hat ($\hat{x}$)**: Unit vector along the **C-N bond direction**
   - Points from the bonded carbon to the nitrogen atom
   - Represents the direction of the molecular dipole moment

2. **z_hat ($\hat{z}$)**: Unit vector **perpendicular to the benzene ring plane**
   - Defines the plane of the aromatic ring
   - Computed using the cross product of two in-plane vectors

These vectors allow us to measure the molecule's orientation relative to the applied electric field at each timestep.

In [ ]:
# Compute x_hat: unit vector along C-N bond
# Points from bonded carbon toward nitrogen
x_hat = para_parsed["N"][0] - para_parsed["C"][para_C_index]
x_hat = x_hat / np.linalg.norm(x_hat)

# Compute z_hat: unit vector perpendicular to ring plane
# Use two carbon-carbon vectors in the ring (C0->C1 and C0->C2)
c1_vec = para_parsed["C"][1] - para_parsed["C"][0]
c2_vec = para_parsed["C"][2] - para_parsed["C"][0]

# Take cross product to get normal to the plane
# Using the formula: n = (v1 × v2) / |v1 × v2|
z_hat = np.cross(c1_vec, c2_vec)
z_hat = z_hat / np.linalg.norm(z_hat)

# Ensure z_hat points in positive z-direction
if z_hat[2] < 0:
    z_hat = -z_hat

print("Molecular orientation vectors:")
print(f"  x_hat (C-N bond):   {x_hat}")
print(f"  z_hat (ring normal): {z_hat}")


Molecular orientation vectors:
  x_hat (C-N bond):   [ 0.99984401  0.01443784 -0.01017342]
  z_hat (ring normal): [0.01035969 0.00235986 0.99994355]


### Field Alignment Analysis

Now we calculate how the molecular vectors are oriented relative to the applied electric field. The field vector is defined by its components along the laboratory x, y, z axes.

We compute two key angles:
- **$\phi$ (phi)**: Angle between the C-N bond (x_hat) and the field
- **$\theta$ (theta)**: Angle between the ring normal (z_hat) and the field

These angles tell us how the molecule is oriented in the field and will change as the molecule rotates during the simulation.

In [ ]:
# Define the electric field vector (laboratory frame)
field_vector = np.array([0.7839420737139418, 0.5571186332860504, 0.27395921869243256])
field_vector = field_vector / np.linalg.norm(field_vector)  # Normalize

# Compute angle between C-N bond and field
cos_phi = np.dot(x_hat, field_vector)
phi_deg = np.degrees(np.arccos(np.clip(cos_phi, -1, 1)))  # Clip for numerical stability

# Compute angle between ring normal and field
cos_theta = np.dot(z_hat, field_vector)
theta_deg = np.degrees(np.arccos(np.clip(cos_theta, -1, 1)))

print("\nField alignment:")
print(f"  C-N bond to field:   φ = {phi_deg:.2f}° (cos φ = {cos_phi:.4f})")
print(f"  Ring normal to field: θ = {theta_deg:.2f}° (cos θ = {cos_theta:.4f})")


Field alignment:
  C-N bond to field:   φ = 37.90° (cos φ = 0.7891)
  Ring normal to field: θ = 73.54° (cos θ = 0.2834)


### Interpreting the Angles

- **$\phi \approx 0$°**: C-N bond aligned parallel with field (maximum alignment between C-N bond and field vector)
- **$\phi \approx 90$°**: C-N bond perpendicular to field
- **$\theta \approx  0$°**: Ring plane perpendicular to field (minimum alignment between field and molecule)
- **$\theta \approx 90$°**: Ring plane parallel to field (field maximally aligned in the plane of the molecule)

The electric field will exert a torque on the molecule that tends to re-orient the molecule. During dynamics, you can track how these angles evolve over time to understand the rotational motion of these two vectors with respect to the field vector.

## Defining the Electric Field Vector

The electric field direction is specified using **spherical coordinates**:
- **$\theta$ (theta)**: Polar angle from the z-axis (0° = along +z, 90° = in xy-plane, 180° = along -z)
- **$\phi$ (phi)**: Azimuthal angle in the xy-plane from the x-axis (0° = along +x, 90° = along +y)

This parameterization is convenient because:
1. The field *direction* is independent of magnitude (we only care about orientation)
2. $\theta$ and $\phi$ can be systematically varied to explore different field orientations
3. It matches the natural coordinate system for rotational dynamics

The function converts ($\theta$, $\phi$) → ($x$, $y$, $z$) Cartesian coordinates for the field vector.

In [ ]:
def generate_field_vector_from_theta_and_phi(theta, phi):
    """
    Generate a unit field vector from spherical coordinates.

    Parameters:
    -----------
    theta : float
        Polar angle in degrees (0° = +z axis, 90° = xy-plane, 180° = -z axis)
    phi : float
        Azimuthal angle in degrees (0° = +x axis, 90° = +y axis)

    Returns:
    --------
    array : Field vector [x, y, z] as a unit vector

    Spherical to Cartesian conversion:
        x = sin(θ) cos(φ)
        y = sin(θ) sin(φ)
        z = cos(θ)
    """
    # Convert degrees to radians
    theta_rad = np.radians(theta)
    phi_rad = np.radians(phi)

    # Compute Cartesian components
    x = np.sin(theta_rad) * np.cos(phi_rad)
    y = np.sin(theta_rad) * np.sin(phi_rad)
    z = np.cos(theta_rad)

    return np.array([x, y, z])

# Example: Generate field vector
theta = 90.0  # 45° from z-axis
phi = 0.0    # 30° from x-axis in xy-plane

field_vector = generate_field_vector_from_theta_and_phi(theta, phi)
print(f"Field vector for θ={theta}°, φ={phi}°:")
print(f"  [x, y, z] = [{field_vector[0]:.4f}, {field_vector[1]:.4f}, {field_vector[2]:.4f}]")
print(f"  Magnitude: {np.linalg.norm(field_vector):.6f} (should be 1.0)")

Field vector for θ=90.0°, φ=0.0°:
  [x, y, z] = [1.0000, 0.0000, 0.0000]
  Magnitude: 1.000000 (should be 1.0)
